In [1]:
import os
from datetime import datetime

from sqlalchemy import create_engine
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey,DateTime,Boolean,Date,Time,Float,DATE,DATETIME,TIME
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship

import statsapi as mlb
from datetime import datetime
import time

class MyDatabase:
    # http://docs.sqlalchemy.org/en/latest/core/engines.html
    DB_ENGINE = {
       'sqlite': 'sqlite:////{DB}'
    }

    # Main DB Connection Ref Obj
    db_engine = None
    def __init__(self, dbtype, username='', password='', dbname='',path=os.getcwd()+'/'):
        dbtype = dbtype.lower()
        if dbtype in self.DB_ENGINE.keys():
            engine_url = self.DB_ENGINE[dbtype].format(DB=path+dbname)
            self.db_engine = create_engine(engine_url)
            print(self.db_engine)
        else:
            print("DBType is not found in DB_ENGINE")

db = MyDatabase('sqlite',dbname='mlb.db')

from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=db.db_engine)
session = Session()
Base = declarative_base()

Engine(sqlite://///Users/schlinkertc/code/MLB/mlb_sqlite/mlb.db)


In [3]:
db.db_engine.table_names()

['game_team_link', 'games', 'people', 'pitches', 'plays', 'teams']

In [4]:
import sql_alchemy_mapping

Game = sql_alchemy_mapping.Game
Person = sql_alchemy_mapping.Person
Pitch = sql_alchemy_mapping.Pitch
Play = sql_alchemy_mapping.Play
Team = sql_alchemy_mapping.Team
GameTeamLink = sql_alchemy_mapping.GameTeamLink

Engine(sqlite://///Users/schlinkertc/code/MLB/mlb_sqlite/mlb.db)


In [13]:
from os import walk
import csv
import re
def read_gamePks():
    gamePks_path = os.getcwd()+"/gamePks"
    f = []
    for (dirpath, dirnames, filenames) in walk(gamePks_path):
        f.extend(filenames)
        break
    pk_paths = [gamePks_path + '/' + x for x in f if x[0]!= '.']
    
    gamePks = {}
    for path in pk_paths:
        season = re.findall('/gamePks/([^.csv]+)',path)
        with open(path, 'r') as f:
            reader = csv.reader(f)
            seasonPks = list(reader)
        gamePks[season[0]] = [item for sublist in seasonPks for item in sublist]
    return gamePks

import numpy as np
import random 
gamePk_dict=read_gamePks()
games_2019 = gamePk_dict['2019']

games = random.sample(games_2019,len(games_2019)//10)

In [7]:
count = 1 
for game in games:
    print(f'starting {count} out of {len(games)}')
    Game(game,session,commit=True,verbose=True)
    count+=1

starting 1 out of 300
adding game record
adding play records
commit...
starting 2 out of 300
adding game record
adding play records
commit...
starting 3 out of 300
adding game record
adding play records
commit...
starting 4 out of 300
adding game record
adding play records
commit...
starting 5 out of 300
adding game record
adding play records
commit...
starting 6 out of 300
adding game record
adding play records
commit...
starting 7 out of 300
adding game record
adding play records
commit...
starting 8 out of 300
adding game record
adding play records
commit...
starting 9 out of 300
adding game record
adding play records
commit...
starting 10 out of 300
adding game record
adding play records
commit...
starting 11 out of 300
adding game record
adding play records
commit...
starting 12 out of 300


KeyboardInterrupt: 

In [11]:
# get the team ids from the games table
# de-dupe against team ids already in the teams table
# add and commit remaining teams

teams = {item for sublist in 
         session.execute('select homeTeam_id,awayTeam_id from games').fetchall() 
         for item in sublist}

already_added = {item for sublist in 
         session.execute('select id from teams').fetchall() 
         for item in sublist}

teams = list(teams-already_added)

session.add_all(Team(x) for x in teams)

session.commit()

In [6]:
game_ids=session.execute("""select id from games""").fetchall()
game_ids=[item for sublist in game_ids for item in sublist]


In [8]:
def chunk(n,list_to_chunk):
    """
    takes in n, and a list to chunk. returns a list of lists with n length. The last chunk size may or may not 
    be equal to n. 
    """
    return [ list_to_chunk[i:i+n] for i in range(0,len(list_to_chunk),n) ]

In [9]:
chunked_game_ids = chunk(30,game_ids)
count=1

for ids in chunked_game_ids:
    print(f'chunk{count} out of {len(chunked_game_ids)}')
    session.add_all([GameTeamLink(x,'away') for x in ids])
    session.commit()
    count+=1
    

chunk1 out of 1
